\
            # 52. 框架基础：Flask（Flask Fundamentals）

            目标：快速掌握 Flask 的核心概念：路由、请求/响应、JSON、错误处理、配置、蓝图（Blueprint）与测试思路。
本章示例对 Flask 做可选依赖处理：未安装时给出安装指引；安装后可在本机线程里启动服务并用 urllib 调用。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- HTTP/REST 基础
- Python 函数/异常/模块基础


## 知识点地图

- 1. Flask 是什么：轻量 WSGI Web 框架
- 2. 最小应用：路由 + JSON 响应
- 3. 请求与响应：headers、query、body、cookie（常用）
- 4. 错误处理：errorhandler + 统一错误格式
- 5. 应用结构：Blueprint（模块化）与工厂模式（概念）
- 6. 测试：test_client（了解）
- 7. 部署与生产（概念）：不要用开发服务器上线


## 自检清单（学完打勾）

- [ ] 会创建 Flask app 并定义路由（GET/POST）
- [ ] 理解 request/response 与 JSON 处理
- [ ] 会写错误处理（errorhandler）与统一错误格式
- [ ] 知道配置的来源（config/env）与开发/生产差异（概念）
- [ ] 理解 Blueprint 用于模块化组织代码
- [ ] 会用 Flask test_client 做基础接口测试（了解）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：Flask 是什么：轻量 WSGI Web 框架

- Flask 基于 WSGI：一次请求对应一次函数调用，返回响应。
- 适合：中小型服务、原型、服务端渲染等。
- 现代 Python Web 里，Flask 更偏“同步模型”，高并发场景常配合 WSGI server（gunicorn/uwsgi）与多进程/多线程。


## 知识点 2：最小应用：路由 + JSON 响应

- 路由：`@app.get('/path')` / `@app.post('/path')`
- JSON：`flask.jsonify` 生成带正确 Content-Type 的响应

安装：`pip install flask`


In [ ]:
import threading
import time
from urllib.request import urlopen, Request

try:
    from flask import Flask, jsonify, request
    from werkzeug.serving import make_server
except Exception as e:
    print('flask not available:', type(e).__name__, e)
    print('install: pip install flask')
else:
    app = Flask(__name__)

    @app.get('/ping')
    def ping():
        return jsonify({'ok': True})

    @app.post('/echo')
    def echo():
        data = request.get_json(silent=True) or {}
        return jsonify({'you_sent': data})

    server = make_server('127.0.0.1', 0, app)
    host, port = server.server_address
    t = threading.Thread(target=server.serve_forever, daemon=True)
    t.start()

    with urlopen(f'http://{host}:{port}/ping', timeout=2) as resp:
        print('GET /ping ->', resp.status, resp.read().decode('utf-8'))

    req = Request(f'http://{host}:{port}/echo', method='POST', data=b'{"x": 1}')
    req.add_header('Content-Type', 'application/json')
    with urlopen(req, timeout=2) as resp:
        print('POST /echo ->', resp.status, resp.read().decode('utf-8'))

    server.shutdown()


## 知识点 3：请求与响应：headers、query、body、cookie（常用）

- query：`request.args.get('q')`
- header：`request.headers.get('X-...')`
- body：`request.get_json()`（JSON） / `request.form`（表单）
- response：可以返回 (body, status_code, headers)

建议：为接口定义统一的错误结构与 request_id（与 RESTful 章呼应）。


## 知识点 4：错误处理：errorhandler + 统一错误格式

统一错误格式能让客户端稳定处理异常：
- 参数错误 400
- 资源不存在 404
- 未处理异常 500

Flask 支持：
- `@app.errorhandler(ExceptionType)`
- 自定义异常类（带 code/message）


In [ ]:
import threading
from urllib.request import urlopen

try:
    from flask import Flask, jsonify
    from werkzeug.serving import make_server
except Exception as e:
    print('flask not available:', type(e).__name__, e)
else:
    class ApiError(Exception):
        def __init__(self, code: str, message: str, status: int = 400):
            self.code = code
            self.message = message
            self.status = status

    app = Flask(__name__)

    @app.get('/boom')
    def boom():
        raise ApiError('BAD', 'something wrong', status=400)

    @app.errorhandler(ApiError)
    def handle_api_error(e: ApiError):
        return jsonify({'error': {'code': e.code, 'message': e.message}}), e.status

    server = make_server('127.0.0.1', 0, app)
    host, port = server.server_address
    threading.Thread(target=server.serve_forever, daemon=True).start()

    try:
        urlopen(f'http://{host}:{port}/boom')
    except Exception as e:
        print('urllib sees error for 4xx, this is expected')

    server.shutdown()


## 知识点 5：应用结构：Blueprint（模块化）与工厂模式（概念）

当项目变大时：
- 用 Blueprint 拆分路由模块（user/order/admin...）。
- 用“应用工厂”创建 app：`create_app(config)`，便于测试与多环境配置。

这能让：
- 依赖注入更清晰（db、cache、mq）
- 测试更容易（不同配置创建不同 app）


## 知识点 6：测试：test_client（了解）

Flask 自带 test_client：不用真的起端口也能测路由。

核心价值：
- 快、稳定、可在 CI 跑
- 覆盖你最关心的：输入校验、状态码、错误结构


## 知识点 7：部署与生产（概念）：不要用开发服务器上线

- `app.run()` 是开发服务器，不适合生产。
- 生产常用：gunicorn/uwsgi + 多 worker；反向代理 Nginx。
- 配合 Docker/日志/监控/健康检查，形成可运维服务。


## 常见坑

- 把开发服务器当生产：性能与稳定性不足
- 不做输入校验：任意 JSON 导致 500 或安全问题
- 错误结构不统一：客户端难处理
- 全局变量滥用：多进程/多线程下状态混乱
- 路由与业务逻辑混在一起：代码难维护


## 综合小案例：用 Flask 写一个最小 REST API（Todo）

要求：
- GET /todos 列表
- POST /todos 创建（参数校验，返回 201）
- GET /todos/<id> 读取
- DELETE /todos/<id> 删除（204）

提示：
- 先用内存 dict 存储
- 统一错误结构（errorhandler）
- 为 POST 增加幂等键（可选）


In [ ]:
# 建议：参考 47_RESTful接口设计.ipynb 的 TodoAPI 思路，用 Flask 重写。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- Flask（WSGI）与 FastAPI（ASGI）在并发模型上有什么差异（概念）？
- 为什么生产环境不建议使用 app.run()？
- 如何在 Flask 里做到“统一错误格式 + 正确状态码”？
- Blueprint 解决了什么问题？


## 练习题（建议写代码）

- 实现 Todo API 并写 5 条 test_client 测试用例（创建/参数错误/404/删除）。
- 加入简单的认证：header token 校验（示意即可）。
- 把数据存储替换为 sqlite3 或 MongoDB（与对应章节呼应）。
